# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 11-3. GTFS Realtime : Vehicle Position

### 1. 下準備

In [1]:
import requests
import zipfile
import os
from google.transit import gtfs_realtime_pb2
import pandas as pd
import geopandas as gpd

In [2]:
AOMORI_BUS_ALERT_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_alert'
AOMORI_BUS_TRIPUPDATE_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_trip_update'
AOMORI_BUS_VEHICLEPOSITION_URL = 'https://api-public.odpt.org/api/v4/gtfs/realtime/odpt_AomoriCity_AllLines_vehicle'

### 2. GTFS Realtime のダウンロード
#### 2-a) フィードの読み込み

In [3]:
response = requests.get(AOMORI_BUS_VEHICLEPOSITION_URL)

if response.status_code == 200:
    pb_data = response.content

In [4]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(pb_data)

4823

#### 2-b) ヘッダとエンティティの表示

In [5]:
print(feed.header)

gtfs_realtime_version: "1.0"
incrementality: FULL_DATASET
timestamp: 1734337700



In [6]:
for entity in feed.entity:
    print(entity)

id: "VP-1"
vehicle {
  trip {
    trip_id: "平日_16時44分_系統17032"
  }
  vehicle {
    id: "1253"
    label: "IchigoBUS"
    license_plate: "1253"
  }
  position {
    latitude: 40.8225441
    longitude: 140.750259
  }
  current_stop_sequence: 22
  current_status: IN_TRANSIT_TO
  timestamp: 1734337660
}

id: "VP-2"
vehicle {
  trip {
    trip_id: "平日_17時15分_系統16031"
  }
  vehicle {
    id: "1397"
    label: "IchigoBUS"
    license_plate: "1397"
  }
  position {
    latitude: 40.8231163
    longitude: 140.746918
  }
  current_stop_sequence: 5
  current_status: IN_TRANSIT_TO
  timestamp: 1734337672
}

id: "VP-3"
vehicle {
  trip {
    trip_id: "平日_16時30分_系統80031"
  }
  vehicle {
    id: "1106"
    label: "IchigoBUS"
    license_plate: "1106"
  }
  position {
    latitude: 40.817955
    longitude: 140.750854
  }
  current_stop_sequence: 19
  current_status: IN_TRANSIT_TO
  timestamp: 1734337644
}

id: "VP-4"
vehicle {
  trip {
    trip_id: "平日_16時49分_系統39082"
  }
  vehicle {
    id: "1105"
  

#### 2-c) Pandas での読み込み

In [7]:
data = []
for entity in feed.entity:
    if not entity.HasField('vehicle'):
        continue
    
    vehicle = entity.vehicle    
    data.append({
        'vhiecle_id': vehicle.vehicle.id,
        'trip_id': vehicle.trip.trip_id,
        'current_stop_sequence': vehicle.current_stop_sequence,
        'cuttent_status': vehicle.current_status,
        'lat': vehicle.position.latitude,
        'lon': vehicle.position.longitude
    })

df_vehicle = pd.DataFrame(data)
df_vehicle

,vhiecle_id,trip_id,current_stop_sequence,cuttent_status,lat,lon
0,1253,平日_16時44分_系統17032,22,2,40.822544,140.750259
1,1397,平日_17時15分_系統16031,5,2,40.823116,140.746918
2,1106,平日_16時30分_系統80031,19,2,40.817955,140.750854
3,1105,平日_16時49分_系統39082,21,2,40.824684,140.764404
4,1252,平日_16時52分_系統20131,20,2,40.770744,140.745453
5,0727,平日_17時20分_系統24011,4,2,40.823814,140.740173
6,1447,平日_17時25分_系統15032,2,2,40.819283,140.808960
7,1040,平日_17時15分_系統28102,6,2,40.802860,140.740463
8,0719,平日_16時36分_系統21082,24,2,40.856247,140.693954
9,0728,平日_17時11分_系統6161,9,2,40.817345,140.755692


#### 2-d) GeoPandas での読み込み

In [8]:
gdf_vehicle = gpd.GeoDataFrame(df_vehicle, geometry=gpd.points_from_xy(df_vehicle['lon'], df_vehicle['lat']), crs=4326)
gdf_vehicle 

,vhiecle_id,trip_id,current_stop_sequence,cuttent_status,lat,lon,geometry
0,1253,平日_16時44分_系統17032,22,2,40.822544,140.750259,POINT (140.75026 40.82254)
1,1397,平日_17時15分_系統16031,5,2,40.823116,140.746918,POINT (140.74692 40.82312)
2,1106,平日_16時30分_系統80031,19,2,40.817955,140.750854,POINT (140.75085 40.81796)
3,1105,平日_16時49分_系統39082,21,2,40.824684,140.764404,POINT (140.76440 40.82468)
4,1252,平日_16時52分_系統20131,20,2,40.770744,140.745453,POINT (140.74545 40.77074)
5,0727,平日_17時20分_系統24011,4,2,40.823814,140.740173,POINT (140.74017 40.82381)
6,1447,平日_17時25分_系統15032,2,2,40.819283,140.808960,POINT (140.80896 40.81928)
7,1040,平日_17時15分_系統28102,6,2,40.802860,140.740463,POINT (140.74046 40.80286)
8,0719,平日_16時36分_系統21082,24,2,40.856247,140.693954,POINT (140.69395 40.85625)
9,0728,平日_17時11分_系統6161,9,2,40.817345,140.755692,POINT (140.75569 40.81734)


In [9]:
gdf_vehicle.explore()

### 3. GTFSの読み込み
#### 3-a) GTFSのダウンロードと展開

In [10]:
AOMORI_BUS_GTFS = 'https://api-public.odpt.org/api/v4/files/odpt/AomoriCity/AllLines.zip?date=20241201'

ZIP_PATH = 'tmp.zip'
TMPDIR_PATH = 'tmp/'

In [11]:
response = requests.get(AOMORI_BUS_GTFS)

if response.status_code == 200:
  with open(ZIP_PATH, 'wb') as f:
    f.write(response.content)

if not os.path.exists(TMPDIR_PATH):
  os.makedirs(TMPDIR_PATH)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
  zip_ref.extractall(TMPDIR_PATH)

#### 3-b) stops.txt / routes.txt / trips.txt / stop_times.txt の読み込み

In [12]:
df_stops = pd.read_csv(TMPDIR_PATH + 'stops.txt')
df_stops = df_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
df_stops

,stop_id,stop_name,stop_lat,stop_lon
0,1_01,青森駅 (降車専用),40.828081,140.734870
1,1_02,青森駅 ②のりば,40.828131,140.734587
2,1_03,青森駅 ③のりば,40.828027,140.734523
3,1_04,青森駅 ④のりば,40.827924,140.734468
4,1_06,青森駅 ⑥のりば,40.828250,140.734850
...,...,...,...,...
817,864_01,大野中央公園前,40.800569,140.726947
818,867_01,安田,40.804843,140.708184
819,867_02,安田,40.805341,140.708471
820,263_01,総合体育館前,40.813405,140.746322


In [13]:
df_routes = pd.read_csv(TMPDIR_PATH + 'routes.txt')
df_routes = df_routes[['route_id', 'route_short_name', 'route_long_name']]
df_routes

,route_id,route_short_name,route_long_name
0,国道・古川線(1021),A1 国道・古川線,NaN
1,造道・八重田線(1022),C10 造道・八重田線,NaN
2,造道・八重田線(1041),C12 造道・八重田線,NaN
3,国道・古川線(1051),A1 国道・古川線,NaN
4,造道・八重田線(1052),C10 造道・八重田線,NaN
...,...,...,...
251,沖館・新田線(42052),W53 沖館・新田線,NaN
252,浪館中央循環(82031),浪館中央循環,NaN
253,浪館中央循環(82032),浪館中央循環,NaN
254,旭町通り線(18001),M36 旭町通り線,NaN


In [14]:
df_trips = pd.read_csv(TMPDIR_PATH + 'trips.txt')
df_trips = df_trips[['route_id', 'service_id', 'trip_id', 'trip_headsign']]
df_trips

,route_id,service_id,trip_id,trip_headsign
0,国道・古川線(1051),平日,平日_05時50分_系統1051,A1青森駅（国道・古川線）
1,造道・八重田線(39202),平日,平日_06時00分_系統39202,C10県病→東部〈営〉（造道・八重田線）
2,石江・新城線(39081),平日,平日_06時00分_系統39081,T50西部〈営〉（石江・新城線）
3,浪館通り線(35141),平日,平日_06時05分_系統35141,P40慈恵会病院（浪館通り線）
4,国道・古川線(1051),平日,平日_06時07分_系統1051,A1青森駅（国道・古川線）
...,...,...,...,...
1607,石江・新城線(16052),土日祝,土日祝_21時45分_系統16052,T50西部〈営〉（石江・新城線）
1608,造道・八重田線(1052),土日祝,土日祝_21時45分_系統1052,C10古川→東部〈営〉（造道・八重田線）
1609,観光通り線(9031),土日祝,土日祝_21時52分_系統9031,K32幸畑団地（観光通り線）
1610,石江・新城線(39051),土日祝,土日祝_21時52分_系統39051,T50西部〈営〉（石江・新城線）


In [15]:
df_stop_times = pd.read_csv(TMPDIR_PATH + 'stop_times.txt')
df_stop_times = df_stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_headsign']]
df_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign
0,平日_05時50分_系統1051,05:50:00,05:50:00,726_01,1,♿ A1青森駅（国道・古川線）
1,平日_05時50分_系統1051,05:51:00,05:51:00,53_01,2,♿ A1青森駅（国道・古川線）
2,平日_05時50分_系統1051,05:51:00,05:51:00,52_01,3,♿ A1青森駅（国道・古川線）
3,平日_05時50分_系統1051,05:52:00,05:52:00,51_01,4,♿ A1青森駅（国道・古川線）
4,平日_05時50分_系統1051,05:53:00,05:53:00,50_01,5,♿ A1青森駅（国道・古川線）
...,...,...,...,...,...,...
44666,土日祝_22時05分_系統1052,22:30:00,22:30:00,50_02,21,♿ C10東部〈営〉（造道・八重田線）
44667,土日祝_22時05分_系統1052,22:31:00,22:31:00,51_02,22,♿ C10東部〈営〉（造道・八重田線）
44668,土日祝_22時05分_系統1052,22:31:00,22:31:00,52_02,23,♿ C10東部〈営〉（造道・八重田線）
44669,土日祝_22時05分_系統1052,22:32:00,22:32:00,53_02,24,♿ C10東部〈営〉（造道・八重田線）


#### 3-c) 特定の路線での絞り込み

In [16]:
df_routes_a1 = df_routes[df_routes['route_short_name'] == 'A1 国道・古川線']
df_trips_a1 = df_routes_a1.merge(df_trips)
df_trips_a1

,route_id,route_short_name,route_long_name,service_id,trip_id,trip_headsign
0,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_06時28分_系統1021,A1青森駅（国道・古川線）
1,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_06時59分_系統1021,A1青森駅（国道・古川線）
2,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_07時30分_系統1021,A1青森駅（国道・古川線）
3,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_07時47分_系統1021,A1青森駅（国道・古川線）
4,国道・古川線(1021),A1 国道・古川線,NaN,平日,平日_08時15分_系統1021,A1青森駅（国道・古川線）
...,...,...,...,...,...,...
432,国道・古川線(12212),A1 国道・古川線,NaN,土日祝,土日祝_09時10分_系統12212,A1中央大橋→青森駅（国道・古川線）
433,国道・古川線(16152),A1 国道・古川線,NaN,平日,平日_13時00分_系統16152,A1中央大橋→青森駅（国道・古川線）
434,国道・古川線(16152),A1 国道・古川線,NaN,土日祝,土日祝_13時04分_系統16152,A1中央大橋→青森駅（国道・古川線）
435,国道・古川線(16152),A1 国道・古川線,NaN,土日祝,土日祝_18時22分_系統16152,A1中央大橋→青森駅（国道・古川線）


#### 3-d) 特定の路線上の車両位置情報を確認

In [17]:
gdf_vehicle_a1 = gdf_vehicle.merge(df_trips_a1)
gdf_vehicle_a1

,vhiecle_id,trip_id,current_stop_sequence,cuttent_status,lat,lon,geometry,route_id,route_short_name,route_long_name,service_id,trip_headsign
0,1253,平日_16時44分_系統17032,22,2,40.822544,140.750259,POINT (140.75026 40.82254),国道・古川線(17032),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）
1,1040,平日_17時15分_系統28102,6,2,40.802860,140.740463,POINT (140.74046 40.80286),国道・古川線(28102),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）
2,1339,平日_17時12分_系統22022,8,2,40.791424,140.760574,POINT (140.76057 40.79142),国道・古川線(22022),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）
3,1388,平日_17時07分_系統47032,27,2,40.870857,140.683533,POINT (140.68353 40.87086),国道・古川線(47032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）
4,0729,平日_17時25分_系統44374,1,2,40.815659,140.687424,POINT (140.68742 40.81566),国道・古川線(44374),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）
5,1446,平日_17時05分_系統17002,13,2,40.790443,140.761032,POINT (140.76103 40.79044),国道・古川線(17002),A1 国道・古川線,NaN,平日,A1中央大橋→青森駅（国道・古川線）
6,1347,平日_17時15分_系統6162,11,2,40.827137,140.794983,POINT (140.79498 40.82714),国道・古川線(6162),A1 国道・古川線,NaN,平日,A1小柳団地→青森駅（国道・古川線）
7,1256,平日_17時20分_系統16042,6,2,40.776859,140.765823,POINT (140.76582 40.77686),国道・古川線(16042),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）
8,1310,平日_17時24分_系統56061,5,2,40.838768,140.803009,POINT (140.80301 40.83877),国道・古川線(56061),A1 国道・古川線,NaN,平日,A1東バイパス→青森駅（国道・古川線）
9,1309,平日_17時18分_系統37032,5,2,40.808125,140.736938,POINT (140.73694 40.80812),国道・古川線(37032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）


In [18]:
df_stop_times_renamed = df_stop_times.rename(columns={'stop_sequence' : 'current_stop_sequence'})
df_stops_names = df_stops[['stop_id', 'stop_name']]
gdf_vehicle_a1.merge(df_stop_times_renamed).merge(df_stops_names)

,vhiecle_id,trip_id,current_stop_sequence,cuttent_status,lat,lon,geometry,route_id,route_short_name,route_long_name,service_id,trip_headsign,arrival_time,departure_time,stop_id,stop_headsign,stop_name
0,1253,平日_16時44分_系統17032,22,2,40.822544,140.750259,POINT (140.75026 40.82254),国道・古川線(17032),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）,17:17:00,17:17:00,6_03,A1青森駅（国道・古川線）,ＮＴＴ青森支店前
1,1040,平日_17時15分_系統28102,6,2,40.802860,140.740463,POINT (140.74046 40.80286),国道・古川線(28102),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）,17:20:00,17:20:00,757_02,♿ A1旭町→青森駅（国道・古川線）,大野若宮
2,1339,平日_17時12分_系統22022,8,2,40.791424,140.760574,POINT (140.76057 40.79142),国道・古川線(22022),A1 国道・古川線,NaN,平日,A1観光通り→青森駅（国道・古川線）,17:20:00,17:20:00,244_02,♿ A1観光通り→青森駅（国道・古川線）,妙見
3,1388,平日_17時07分_系統47032,27,2,40.870857,140.683533,POINT (140.68353 40.87086),国道・古川線(47032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:23:00,17:23:00,440_02,♿ A1青森駅（国道・古川線）,夏井田
4,0729,平日_17時25分_系統44374,1,2,40.815659,140.687424,POINT (140.68742 40.81566),国道・古川線(44374),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:25:00,17:25:00,359_03,♿ A1青森駅（国道・古川線）,つくしが丘病院前
5,1446,平日_17時05分_系統17002,13,2,40.790443,140.761032,POINT (140.76103 40.79044),国道・古川線(17002),A1 国道・古川線,NaN,平日,A1中央大橋→青森駅（国道・古川線）,17:23:00,17:23:00,247_02,♿ A1中央大橋→青森駅（国道・古川線）,問屋町入口
6,1347,平日_17時15分_系統6162,11,2,40.827137,140.794983,POINT (140.79498 40.82714),国道・古川線(6162),A1 国道・古川線,NaN,平日,A1小柳団地→青森駅（国道・古川線）,17:26:00,17:26:00,46_01,A1小柳団地→青森駅（国道・古川線）,県立中央病院通り
7,1256,平日_17時20分_系統16042,6,2,40.776859,140.765823,POINT (140.76582 40.77686),国道・古川線(16042),A1 国道・古川線,NaN,平日,A1旭町→青森駅（国道・古川線）,17:25:00,17:25:00,253_02,♿ A1旭町→青森駅（国道・古川線）,横内
8,1310,平日_17時24分_系統56061,5,2,40.838768,140.803009,POINT (140.80301 40.83877),国道・古川線(56061),A1 国道・古川線,NaN,平日,A1東バイパス→青森駅（国道・古川線）,17:27:00,17:27:00,50_01,A1東バイパス→青森駅（国道・古川線）,原別
9,1309,平日_17時18分_系統37032,5,2,40.808125,140.736938,POINT (140.73694 40.80812),国道・古川線(37032),A1 国道・古川線,NaN,平日,A1青森駅（国道・古川線）,17:25:00,17:25:00,333_02,A1青森駅（国道・古川線）,金沢小学校通り
